In [1]:
from langchain_community.vectorstores import FAISS
from langchain_cohere import CohereEmbeddings
from langchain.document_loaders import DataFrameLoader
import time
import pandas as pd
from pydantic import BaseModel, Field
from typing import Literal
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()



True

In [2]:
hadith_df = pd.read_csv("../data/bukhari_hadiths_df.csv")
hadith_df.head(2)


,id,hadith_id,source,chapter_no,hadith_no,text_ar,text_en,chapter_en,chapter_ar
0,0,1,SahihBukhari,1,1,حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سف...,Narrated 'Umar bin Al-Khattab: ...,Revelation,بدء الوحى
1,1,2,SahihBukhari,1,2,حدثنا عبد الله بن يوسف، قال أخبرنا مالك، عن هش...,Narrated 'Aisha: ...,Revelation,بدء الوحى


In [3]:
ar_hadiths = hadith_df['text_ar'].tolist()
print(ar_hadiths[0])

حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سفيان، قال حدثنا يحيى بن سعيد الأنصاري، قال أخبرني محمد بن إبراهيم التيمي، أنه سمع علقمة بن وقاص الليثي، يقول سمعت عمر بن الخطاب  رضى الله عنه  على المنبر قال سمعت رسول الله صلى الله عليه وسلم يقول ‏"‏ إنما الأعمال بالنيات، وإنما لكل امرئ ما نوى، فمن كانت هجرته إلى دنيا يصيبها أو إلى امرأة ينكحها فهجرته إلى ما هاجر إليه ‏"‏‏.‏


In [22]:
# from typing import Optional
# from pydantic import BaseModel, Field
# from langchain_core.output_parsers import PydanticOutputParser
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_openai import ChatOpenAI
# # or: from langchain_cohere import ChatCohere

# # ---------------------------
# # 1. Pydantic Model
# # ---------------------------

# class HadithParts(BaseModel):
#     sanad: str = Field(description="The chain of narrators leading to the Prophet ﷺ")
#     matn: str = Field(description="The actual content spoken by the Prophet ﷺ")

# # ---------------------------
# # 2. LLM initialization
# # ---------------------------

# llm = ChatOpenAI(model="gpt-5-mini", temperature=0)

# # ---------------------------
# # 3. Few-shot examples
# # ---------------------------

# examples = [
#     {
#         "hadith": "حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سفيان، قال حدثنا يحيى بن سعيد الأنصاري، قال أخبرني محمد بن إبراهيم التيمي، أنه سمع علقمة بن وقاص الليثي، يقول سمعت عمر بن الخطاب  رضى الله عنه  على المنبر قال سمعت رسول الله صلى الله عليه وسلم يقول \"إنما الأعمال بالنيات، وإنما لكل امرئ ما نوى، فمن كانت هجرته إلى دنيا يصيبها أو إلى امرأة ينكحها فهجرته إلى ما هاجر إليه\"",
#         "sanad": "حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سفيان، قال حدثنا يحيى بن سعيد الأنصاري، قال أخبرني محمد بن إبراهيم التيمي، أنه سمع علقمة بن وقاص الليثي، يقول سمعت عمر بن الخطاب رضى الله عنه على المنبر قال سمعت رسول الله صلى الله عليه وسلم يقول",
#         "matn": "إنما الأعمال بالنيات، وإنما لكل امرئ ما نوى، فمن كانت هجرته إلى دنيا يصيبها أو إلى امرأة ينكحها فهجرته إلى ما هاجر إليه"
#     },
#     {
#         "hadith": "حدثنا مسلم، قال حدثنا حجاج، عن يونس، عن ابن شهاب، عن عبد الله بن عمر، قال: سمعت رسول الله ﷺ يقول: \"لا ضرر ولا ضرار\"",
#         "sanad": "حدثنا مسلم، قال حدثنا حجاج، عن يونس، عن ابن شهاب، عن عبد الله بن عمر",
#         "matn": "سمعت رسول الله ﷺ يقول: لا ضرر ولا ضرار"
#     },
#     {
#         "hadith": "حدثنا سعيد بن يحيى بن سعيد القرشي، قال حدثنا أبي قال، حدثنا أبو بردة بن عبد الله بن أبي بردة، عن أبي بردة، عن أبي موسى  رضى الله عنه  قال قالوا يا رسول الله أى الإسلام أفضل قال \"من سلم المسلمون من لسانه ويده\"",
#         "sanad": "حدثنا سعيد بن يحيى بن سعيد القرشي، قال حدثنا أبي قال، حدثنا أبو بردة بن عبد الله بن أبي بردة، عن أبي بردة، عن أبي موسى رضى الله عنه",
#         "matn": "قالوا يا رسول الله أى الإسلام أفضل قال من سلم المسلمون من لسانه ويده"
#     }
# ]


# # ---------------------------
# # 4. Parser & Prompt
# # ---------------------------

# parser = PydanticOutputParser(pydantic_object=HadithParts)

# # Format examples as JSON strings inside the prompt
# examples_str = "\n".join([f"- hadith: \"{ex['hadith']}\"\n  sanad: \"{ex['sanad']}\"\n  matn: \"{ex['matn']}\"" for ex in examples])

# prompt = ChatPromptTemplate.from_messages([
#     ("system", f"""You are an expert in Hadith sciences. Your task is to extract the **sanad** (chain of narrators) and **matn** (content) from any hadith text.

# Below are some examples:

# {examples_str}

# Guidelines:
# - The sanad must include all narrators exactly as in the text.
# - The matn must include only the Prophet ﷺ's words.
# - Do NOT add, remove, or modify text.
# - Return your answer strictly in JSON format like the Pydantic schema.

# {{format_instructions}}
# """),("human", "Hadith text:\n{text}")]).partial(format_instructions=parser.get_format_instructions())

# # ---------------------------
# # 5. Function to extract sanad & matn
# # ---------------------------

# def split_hadith(text: str) -> HadithParts:
#     """Split a hadith into sanad and matn using an LLM and Pydantic output parsing."""
#     result = (prompt | llm | parser).invoke({"text": text})
#     return result



In [4]:
from typing import Optional, List
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


# ---------------------------
# 1. Pydantic Model
# ---------------------------

class HadithParts(BaseModel):
    sanad: str
    upper_narrator: str
    matn: str


# ---------------------------
# 2. Example dataset
# ---------------------------

examples = [
    {
        "hadith": "حدثنا مسلم، قال حدثنا حجاج، عن يونس، عن ابن شهاب، عن عبد الله بن عمر، قال: سمعت رسول الله ﷺ يقول: \"لا ضرر ولا ضرار\"",
        "sanad": "حدثنا مسلم، قال حدثنا حجاج، عن يونس، عن ابن شهاب، عن عبد الله بن عمر",
        "upper_narrator": "عبد الله بن عمر",
        "matn": "عن عبد الله بن عمر، قال: سمعت رسول الله ﷺ يقول: لا ضرر ولا ضرار"
    },
    {
        "hadith": "حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سفيان، قال حدثنا يحيى بن سعيد الأنصاري، قال أخبرني محمد بن إبراهيم التيمي، أنه سمع علقمة بن وقاص الليثي، يقول سمعت عمر بن الخطاب رضى الله عنه على المنبر قال سمعت رسول الله ﷺ يقول: \"إنما الأعمال بالنيات، وإنما لكل امرئ ما نوى...\"",
        "sanad": "حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سفيان، قال حدثنا يحيى بن سعيد الأنصاري، قال أخبرني محمد بن إبراهيم التيمي، أنه سمع علقمة بن وقاص الليثي، يقول سمعت عمر بن الخطاب",
        "upper_narrator": "عمر بن الخطاب",
        "matn": "عن عمر بن الخطاب رضي الله عنه قال: سمعت رسول الله ﷺ يقول: إنما الأعمال بالنيات، وإنما لكل امرئ ما نوى"
    },
    {
        "hadith": "حدثنا سعيد بن يحيى بن سعيد القرشي، قال حدثنا أبي قال، حدثنا أبو بردة بن عبد الله بن أبي بردة، عن أبي بردة، عن أبي موسى رضى الله عنه قال قالوا يا رسول الله أى الإسلام أفضل؟ قال: \"من سلم المسلمون من لسانه ويده\"",
        "sanad": "حدثنا سعيد بن يحيى بن سعيد القرشي، قال حدثنا أبي، قال حدثنا أبو بردة بن عبد الله بن أبي بردة، عن أبي بردة، عن أبي موسى",
        "upper_narrator": "أبو موسى",
        "matn": "عن أبي موسى رضي الله عنه قال: قالوا يا رسول الله أي الإسلام أفضل؟ قال: من سلم المسلمون من لسانه ويده"
    },
    {
        "hadith": "حدثنا عبد الله بن يوسف، قال أخبرنا مالك، عن عبد الله بن دينار، عن عبد الله بن عمر رضي الله عنهما أن رسول الله ﷺ قال: \"المسلم أخو المسلم، لا يظلمه ولا يسلمه\"",
        "sanad": "حدثنا عبد الله بن يوسف، قال أخبرنا مالك، عن عبد الله بن دينار، عن عبد الله بن عمر",
        "upper_narrator": "عبد الله بن عمر",
        "matn": "عن عبد الله بن عمر رضي الله عنهما قال: أن رسول الله ﷺ قال: المسلم أخو المسلم، لا يظلمه ولا يسلمه"
    },
    {
        "hadith": "حدثنا قتيبة بن سعيد، قال حدثنا ليث، عن ابن شهاب، عن أنس بن مالك، قال: قال رسول الله ﷺ: \"لا يؤمن أحدكم حتى يحب لأخيه ما يحب لنفسه\"",
        "sanad": "حدثنا قتيبة بن سعيد، قال حدثنا ليث، عن ابن شهاب، عن أنس بن مالك",
        "upper_narrator": "أنس بن مالك",
        "matn": "عن أنس بن مالك قال: قال رسول الله ﷺ: لا يؤمن أحدكم حتى يحب لأخيه ما يحب لنفسه"
    }
]


# ---------------------------
# 3. Build few-shot block with escaped curly braces
# ---------------------------

def escape_json_for_prompt(d: dict) -> str:
    """
    Converts a dictionary into escaped JSON block suitable for LangChain prompt.
    Turns { → {{ and } → }}
    """
    import json
    raw = json.dumps(d, ensure_ascii=False, indent=2)
    escaped = raw.replace("{", "{{").replace("}", "}}")
    return escaped


def build_few_shots(examples: List[dict]) -> str:
    text = ""
    for ex in examples:
        escaped_out = escape_json_for_prompt({
            "sanad": ex["sanad"],
            "upper_narrator": ex["upper_narrator"],
            "matn": ex["matn"]
        })

        text += f"""
Example Input:
{ex['hadith']}

Example Output:
{escaped_out}

"""
    return text


few_shots_text = build_few_shots(examples)


# ---------------------------
# 4. Output parser
# ---------------------------

parser = PydanticOutputParser(pydantic_object=HadithParts)


# ---------------------------
# 5. Prompt
# ---------------------------

prompt = ChatPromptTemplate.from_messages([
    ("system", f"""
You are an expert in Hadith sciences. Extract the following:

1. sanad — full narrator chain before the Prophet ﷺ speaks.
2. upper_narrator — the narrator who directly heard the Prophet ﷺ.
3. matn — the prophetic statement starting from the upper narrator.

Guidelines:
- Do NOT rewrite the text.
- Keep sanad and matn exactly as written.
- Upper narrator = last narrator before the Prophet ﷺ quote.

Here are examples to follow:

{few_shots_text}

Return ONLY JSON in this schema:
{{format_instructions}}
"""),
    ("human", "Hadith:\n{text}")
]).partial(format_instructions=parser.get_format_instructions())


# ---------------------------
# 6. LLM
# ---------------------------

llm = ChatOpenAI(model="gpt-5-mini", temperature=0)


# ---------------------------
# 7. Function
# ---------------------------

def split_hadith(text: str) -> HadithParts:
    return (prompt | llm | parser).invoke({"text": text})


In [5]:
hadith = ar_hadiths[11]
pprint(hadith)
parts = split_hadith(hadith)
pprint(parts)

('حدثنا عمرو بن خالد، قال حدثنا الليث، عن يزيد، عن أبي الخير، عن عبد الله بن '
 'عمرو  رضى الله عنهما  أن رجلا، سأل النبي صلى الله عليه وسلم أى الإسلام خير '
 'قال \u200f"\u200f تطعم الطعام، وتقرأ السلام على من عرفت ومن لم تعرف '
 '\u200f"\u200f\u200f.\u200f')
HadithParts(sanad='حدثنا عمرو بن خالد، قال حدثنا الليث، عن يزيد، عن أبي الخير، عن عبد الله بن عمرو', upper_narrator='عبد الله بن عمرو', matn='عن عبد الله بن عمرو  رضى الله عنهما  أن رجلا، سأل النبي صلى الله عليه وسلم أى الإسلام خير قال \u200f"\u200f تطعم الطعام، وتقرأ السلام على من عرفت ومن لم تعرف \u200f"\u200f\u200f.\u200f')


In [11]:
print("SANAD:\n", parts.sanad)
print("\nMATN:\n", parts.upper_narrator)
print("\nMATN:\n", parts.matn)


SANAD:
 حدثنا عمرو بن خالد، قال حدثنا الليث، عن يزيد، عن أبي الخير، عن عبد الله بن عمرو

MATN:
 عبد الله بن عمرو

MATN:
 عن عبد الله بن عمرو  رضى الله عنهما  أن رجلا، سأل النبي صلى الله عليه وسلم أى الإسلام خير قال ‏"‏ تطعم الطعام، وتقرأ السلام على من عرفت ومن لم تعرف ‏"‏‏.‏


### Batch

In [7]:
def split_hadith_batch(hadith_list):
    """
    Run batch extraction for a list of hadith texts.
    Returns a list of HadithParts objects.
    """

    # Prepare batch input format
    batch_inputs = [{"text": h} for h in hadith_list]

    # Run batch
    results = (prompt | llm | parser).batch(batch_inputs)

    return results

In [13]:
hadiths = ar_hadiths[11:15]
pprint(hadiths)
results = split_hadith_batch(hadiths)
pprint(results)


['حدثنا عمرو بن خالد، قال حدثنا الليث، عن يزيد، عن أبي الخير، عن عبد الله بن '
 'عمرو  رضى الله عنهما  أن رجلا، سأل النبي صلى الله عليه وسلم أى الإسلام خير '
 'قال \u200f"\u200f تطعم الطعام، وتقرأ السلام على من عرفت ومن لم تعرف '
 '\u200f"\u200f\u200f.\u200f',
 'حدثنا مسدد، قال حدثنا يحيى، عن شعبة، عن قتادة، عن أنس  رضى الله عنه  عن '
 'النبي صلى الله عليه وسلم\u200f.\u200fوعن حسين المعلم، قال حدثنا قتادة، عن '
 'أنس، عن النبي صلى الله عليه وسلم قال \u200f"\u200f لا يؤمن أحدكم حتى يحب '
 'لأخيه ما يحب لنفسه \u200f"\u200f\u200f.\u200f',
 'حدثنا أبو اليمان، قال أخبرنا شعيب، قال حدثنا أبو الزناد، عن الأعرج، عن أبي '
 'هريرة  رضى الله عنه  أن رسول الله صلى الله عليه وسلم قال \u200f"\u200f '
 'فوالذي نفسي بيده لا يؤمن أحدكم حتى أكون أحب إليه من والده وولده '
 '\u200f"\u200f\u200f.\u200f',
 'حدثنا يعقوب بن إبراهيم، قال حدثنا ابن علية، عن عبد العزيز بن صهيب، عن أنس، '
 'عن النبي صلى الله عليه وسلم ح وحدثنا آدم، قال حدثنا شعبة، عن قتادة، عن أنس، '
 'قال قال النبي صلى الله عليه وسلم \u200f"\u20

In [18]:
print(results[0])
print(results[0].matn)
print(results[0].sanad)
print(results[0].upper_narrator)

sanad='حدثنا عمرو بن خالد، قال حدثنا الليث، عن يزيد، عن أبي الخير، عن عبد الله بن عمرو' upper_narrator='عبد الله بن عمرو' matn='عن عبد الله بن عمرو  رضى الله عنهما  أن رجلا، سأل النبي صلى الله عليه وسلم أى الإسلام خير قال \u200f"\u200f تطعم الطعام، وتقرأ السلام على من عرفت ومن لم تعرف \u200f"\u200f\u200f.'
عن عبد الله بن عمرو  رضى الله عنهما  أن رجلا، سأل النبي صلى الله عليه وسلم أى الإسلام خير قال ‏"‏ تطعم الطعام، وتقرأ السلام على من عرفت ومن لم تعرف ‏"‏‏.
حدثنا عمرو بن خالد، قال حدثنا الليث، عن يزيد، عن أبي الخير، عن عبد الله بن عمرو
عبد الله بن عمرو


In [ ]:
# import pandas as pd
# from pathlib import Path
# from tqdm import tqdm


# BATCH_SIZE = 50  # عدد الأحاديث لكل batch
# OUTPUT_FILE = "../data/bukhari_hadiths_split_2.csv"

# def process_all_hadith(df, batch_size=BATCH_SIZE, output_file=OUTPUT_FILE):
#     all_results = []

#     # إنشاء ملف جديد أو استكمال إذا موجود
#     output_path = Path(output_file)
#     if output_path.exists():
#         print(f"Appending to existing file {output_file}")
#         existing_df = pd.read_csv(output_file)
#         processed_count = len(existing_df)
#     else:
#         processed_count = 0

#     # حدد unprocessed hadiths
#     hadiths_to_process = df["text_ar"].iloc[processed_count:].tolist()

#     for i in tqdm(range(0, len(hadiths_to_process), batch_size), desc="Processing batches"):
#         batch = hadiths_to_process[i:i+batch_size]
#         batch_results = split_hadith_batch(batch)

#         # تحويل النتائج إلى dicts
#         batch_dicts = [
#             {
#                 "hadith": h,
#                 "sanad": r.sanad,
#                 "upper_narrator": r.upper_narrator,
#                 "matn": r.matn
#             }
#             for h, r in zip(batch, batch_results)
#         ]

#         batch_df = pd.DataFrame(batch_dicts)
#         if output_path.exists():
#             batch_df.to_csv(output_file, mode='a', index=False, header=False)
#         else:
#             batch_df.to_csv(output_file, index=False, header=True)

#         all_results.extend(batch_results)

#     return all_results


# results = process_all_hadith(hadith_df, batch_size=20, output_file=OUTPUT_FILE)


Processing batches:   0%|          | 0/369 [00:00<?, ?it/s]

In [8]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm


BATCH_SIZE = 50
OUTPUT_FILE = "../data/bukhari_hadiths_split.csv"

def process_all_hadith(df, batch_size=BATCH_SIZE, output_file=OUTPUT_FILE):
    
    output_path = Path(output_file)

    if output_path.exists():
        print(f"Appending to existing file: {output_file}")
        existing_df = pd.read_csv(output_file)

        # تحقق من الأعمدة
        required_cols = {"hadith", "sanad", "upper_narrator", "matn"}
        if not required_cols.issubset(set(existing_df.columns)):
            raise ValueError("Existing file does not match required columns!")

        processed_count = len(existing_df)
        print(f"✔ Found existing {processed_count} processed hadiths")
    else:
        processed_count = 0
        print("Starting fresh...")
    
    hadiths_to_process = df["text_ar"].iloc[processed_count:].tolist()
    print(f"➡ Remaining hadiths to process: {len(hadiths_to_process)}")

    all_results = []

    for i in tqdm(range(0, len(hadiths_to_process), batch_size), desc="Processing batches"):
        batch = hadiths_to_process[i:i+batch_size]
        batch_results = split_hadith_batch(batch)

        batch_dicts = [
            {
                "hadith": h,
                "sanad": r.sanad,
                "upper_narrator": r.upper_narrator,
                "matn": r.matn
            }
            for h, r in zip(batch, batch_results)
        ]

        batch_df = pd.DataFrame(batch_dicts)

        if output_path.exists():
            batch_df.to_csv(output_file, mode='a', index=False, header=False)
        else:
            batch_df.to_csv(output_file, index=False, header=True)

        all_results.extend(batch_results)

    return all_results

results = process_all_hadith(hadith_df, batch_size=20, output_file=OUTPUT_FILE)


Appending to existing file: ../data/bukhari_hadiths_split.csv
✔ Found existing 600 processed hadiths
➡ Remaining hadiths to process: 6770


Processing batches:  37%|███▋      | 126/339 [4:05:08<6:54:24, 116.74s/it]


APIConnectionError: Connection error.

In [27]:
hadith_df['text_ar'][0][-100:]

'ا لكل امرئ ما نوى، فمن كانت هجرته إلى دنيا يصيبها أو إلى امرأة ينكحها فهجرته إلى ما هاجر إليه \u200f"\u200f\u200f.\u200f'

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_cohere import CohereEmbeddings
from langchain_community.document_loaders import DataFrameLoader
import time
import pandas as pd
import csv
from typing import Literal, Optional, List
from dotenv import load_dotenv
from pprint import pprint

# Pydantic & LangChain imports
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pathlib import Path
from tqdm import tqdm

load_dotenv()

# Load Data
hadith_df = pd.read_csv("bukhari_hadiths_df.csv")

# ---------------------------
# 1. Pydantic Model
# ---------------------------
class HadithParts(BaseModel):
    sanad: str = Field(description="The full chain of narrators.")
    upper_narrator: str = Field(description="The last narrator who heard directly from the Prophet.")
    matn: str = Field(description="The text of the Hadith starting from the upper narrator.")

# ---------------------------
# 2. Example dataset (Context)
# ---------------------------
examples = [
    {
        "hadith": "حدثنا مسلم... عن عبد الله بن عمر، قال: سمعت رسول الله ﷺ يقول: \"لا ضرر ولا ضرار\"",
        "sanad": "حدثنا مسلم... عن عبد الله بن عمر",
        "upper_narrator": "عبد الله بن عمر",
        "matn": "عن عبد الله بن عمر، قال: سمعت رسول الله ﷺ يقول: لا ضرر ولا ضرار"
    },
    {
        "hadith": "حدثنا عبد الله بن يوسف... عن عبد الله بن عمر رضي الله عنهما أن رسول الله ﷺ قال: \"المسلم أخو المسلم...\"",
        "sanad": "حدثنا عبد الله بن يوسف... عن عبد الله بن عمر",
        "upper_narrator": "عبد الله بن عمر",
        "matn": "عن عبد الله بن عمر رضي الله عنهما قال: أن رسول الله ﷺ قال: المسلم أخو المسلم..."
    }
]

def build_few_shots_text(examples: List[dict]) -> str:
    text = ""
    for ex in examples:
        text += f"Input: {ex['hadith']}\nOutput Sanad: {ex['sanad']}\nOutput Upper: {ex['upper_narrator']}\nOutput Matn: {ex['matn']}\n---\n"
    return text

few_shots_text = build_few_shots_text(examples)

# ---------------------------
# 3. Prompt & LLM Setup
# ---------------------------

# Use the correct model name
llm = ChatOpenAI(model="gpt-5-mini", temperature=0)

# Bind the schema to the LLM (This fixes the Parsing Error)
structured_llm = llm.with_structured_output(HadithParts)

prompt = ChatPromptTemplate.from_messages([
    ("system", f"""
You are an expert in Hadith sciences. Extract the following from the Hadith text:
1. sanad — full narrator chain before the Prophet ﷺ speaks.
2. upper_narrator — the narrator who directly heard the Prophet ﷺ.
3. matn — the prophetic statement starting from the upper narrator.

Guidelines:
- Do NOT rewrite the text.
- Keep sanad and matn exactly as written in Arabic.
- Upper narrator = last narrator before the Prophet ﷺ quote.

Examples of logic:
{few_shots_text}
"""),
    ("human", "Hadith to split:\n{text}")
])

# Create the chain
# Note: output parser is removed because structured_llm returns the Pydantic object directly
chain = prompt | structured_llm

# ---------------------------
# 4. Processing Functions
# ---------------------------

OUTPUT_FILE = "bukhari_hadiths_split_4.csv"
BATCH_SIZE = 50

def process_batch_safe(batch_texts: List[str]) -> List[HadithParts]:
    """
    Tries to process a batch. If the batch fails (e.g., one hadith is too long or weird),
    it falls back to processing item-by-item to save the valid ones.
    """
    batch_inputs = [{"text": h} for h in batch_texts]
    
    try:
        # Try processing the whole batch at once
        return chain.batch(batch_inputs)
    except Exception as e:
        print(f"Batch failed ({str(e)}). Switching to sequential processing for this batch...")
        results = []
        for item in batch_inputs:
            try:
                # Process one by one
                res = chain.invoke(item)
                results.append(res)
            except Exception as inner_e:
                print(f"Skipping error in specific hadith: {inner_e}")
                # Append empty or error placeholder if needed, or just skip
                results.append(HadithParts(sanad="ERROR", upper_narrator="ERROR", matn="ERROR"))
        return results

def process_all_hadith(df, batch_size=BATCH_SIZE, output_file=OUTPUT_FILE):
    output_path = Path(output_file)
    
    # Check for existing progress
    if output_path.exists():
        print(f"Appending to existing file {output_file}")
        existing_df = pd.read_csv(output_file)
        processed_count = len(existing_df)
    else:
        processed_count = 0

    hadiths_to_process = df["text_ar"].iloc[processed_count:].tolist()
    print(f"Total Hadiths: {len(df)}, Remaining: {len(hadiths_to_process)}")

    if not hadiths_to_process:
        print("All hadiths processed.")
        return

    # Loop with TQDM
    for i in tqdm(range(0, len(hadiths_to_process), batch_size), desc="Processing Batches"):
        batch_texts = hadiths_to_process[i:i+batch_size]
        
        # Get results (Pydantic objects)
        batch_results = process_batch_safe(batch_texts)

        # Convert Pydantic objects to dicts
        batch_dicts = []
        for h_text, r in zip(batch_texts, batch_results):
            if r: # Check if result is valid
                batch_dicts.append({
                    "hadith": h_text,
                    "sanad": r.sanad,
                    "upper_narrator": r.upper_narrator,
                    "matn": r.matn
                })

        # Save to CSV immediately
        if batch_dicts:
            batch_df = pd.DataFrame(batch_dicts)
            if output_path.exists():
                batch_df.to_csv(
                    output_file,
                    mode='a',
                    index=False,
                    header=False,
                    sep=',',
                    encoding='utf-8',
                    quoting=csv.QUOTE_MINIMAL,
                    quotechar='"',
                    escapechar='\\',
                    doublequote=True,
                    lineterminator='\n'
                )
            else:
                batch_df.to_csv(
                    output_file,
                    index=False,
                    header=True,
                    sep=',',
                    encoding='utf-8',
                    quoting=csv.QUOTE_MINIMAL,
                    quotechar='"',
                    escapechar='\\',
                    doublequote=True,
                    lineterminator='\n'
                )

# ---------------------------
# 5. Run
# ---------------------------

if __name__ == "__main__":
    process_all_hadith(hadith_df[6411:], batch_size=20, output_file=OUTPUT_FILE)
